In [1]:
manufacturer = 'STANDARD MOTOR PRODUCTS'
type_code = '5061'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()
list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url, part_number in zip(list_url, list_part_number):
    work.put_nowait((url, part_number))

def crawler():
    global df
    
    while not work.empty():
        url, part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                if len(html.xpath('//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]')) != 0:
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Url': url}])
                    
                    status = 'ok'
                    
                    break

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                number_index = np.where(np.array(list_part_number) == part_number)[0]
                
                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]
                
                # = = = = = = = = = = = = = = =
                
                index = list(np.intersect1d(number_index, manufacturer_index))
                
                # = = = = = = = = = = = = = = =
                
                if len(index) == 1:
                    index = index[0]
                    
                    # = = = = = = = = = = = = = = =
                    
                    note_1 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                    if len(note_1) == 0:
                        note_1 = ''
                    else:
                        note_1 = note_1[0].strip()
   
                    # = = = = = = = = = = = = = = =
                    
                    note_2 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]/span/span/text()')
                    if len(note_2) == 0:
                        note_2 = ''
                    else:
                        note_2 = [_.strip() for _ in note_2]
                        while '' in note_2:
                            note_2.remove('')
                        note_2 = '\n'.join(note_2)

                    # = = = = = = = = = = = = = = =
                    
                    oe = list_part[index].xpath('./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]')
                    if len(oe) == 0:
                        oe = ''
                    else:
                        oe = oe[0].xpath('./text()')[0].replace(', ', ';')
                        
                    # = = = = = = = = = = = = = = =
                    
                    src = list_part[index].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                    if len(src) == 0:
                        src = ''
                    else:
                        src = 'https://www.rockauto.com' + src[0].xpath('./@src')[0].replace('__ra_m.', '.')
    
                    # = = = = = = = = = = = = = = =
                    
                    part_json = list_part[index].xpath('./descendant::input[contains(@id, "listing_data_essential")]')
                    if len(part_json) == 0:
                        part_key = ''
                    else:
                        part_key = json.loads(part_json[0].xpath('./@value')[0])['partkey']
                        
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             '序号': '',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Vehcile': '',
                                             'Alternate_OE_Part_Numbers': oe,
                                             'Note_1': note_1,
                                             'Note_2': note_2,
                                             'Pic': '',
                                             'Url': url,
                                             'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + type_code + '&pk=' + part_key,
                                             'Src': src,
                                             'Type_Code': type_code,
                                             'Part_Key': part_key}])
                
                    status = 'ok'
                
                    break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Manufacturer': manufacturer,
                                     'Part_Number': part_number,
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：380

APK103  <->  [ok] - 剩余数量：375
APK102  <->  [ok] - 剩余数量：374
APK100  <->  [ok] - 剩余数量：373
APS103  <->  [ok] - 剩余数量：372
APS102  <->  [ok] - 剩余数量：371
APS106  <->  [ok] - 剩余数量：370
APS104  <->  [ok] - 剩余数量：369
APS107  <->  [ok] - 剩余数量：368
APS105  <->  [ok] - 剩余数量：367


/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_35864/3994342443.py:96: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  number_index = np.where(np.array(list_part_number) == part_number)[0]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_35864/3994342443.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]


APS101  <->  [ok] - 剩余数量：366
APS108  <->  [ok] - 剩余数量：365
APK101  <->  [ok] - 剩余数量：364
APS111  <->  [ok] - 剩余数量：363
APS116  <->  [ok] - 剩余数量：362
APS115  <->  [ok] - 剩余数量：361
APS114  <->  [ok] - 剩余数量：360
APS109  <->  [ok] - 剩余数量：359
APS117  <->  [ok] - 剩余数量：358
APS118  <->  [ok] - 剩余数量：357
APS120  <->  [ok] - 剩余数量：356
APS112  <->  [ok] - 剩余数量：355
APS126  <->  [ok] - 剩余数量：354
APS130  <->  [ok] - 剩余数量：353
APS123  <->  [ok] - 剩余数量：352
APS128  <->  [ok] - 剩余数量：351
APS132  <->  [ok] - 剩余数量：350
APS137  <->  [ok] - 剩余数量：349
APS136  <->  [ok] - 剩余数量：348
APS138  <->  [ok] - 剩余数量：347
APS135  <->  [ok] - 剩余数量：346
APS139  <->  [ok] - 剩余数量：345
APS143  <->  [ok] - 剩余数量：344
APS141  <->  [ok] - 剩余数量：343
APS142  <->  [ok] - 剩余数量：342
APS133  <->  [ok] - 剩余数量：341
APS147  <->  [ok] - 剩余数量：340
APS149  <->  [ok] - 剩余数量：339
APS151  <->  [ok] - 剩余数量：338
APS150  <->  [ok] - 剩余数量：337
APS152  <->  [ok] - 剩余数量：336
APS148  <->  [ok] - 剩余数量：335
APS153  <->  [ok] - 剩余数量：334
APS155  <->  [ok] - 剩余数量：333
APS156  <->  [

APS576  <->  [ok] - 剩余数量：80
APS578  <->  [ok] - 剩余数量：79
APS575  <->  [ok] - 剩余数量：78
APS580  <->  [ok] - 剩余数量：77
APS582  <->  [ok] - 剩余数量：76
APS581  <->  [ok] - 剩余数量：75
APS585  <->  [ok] - 剩余数量：74
APS588  <->  [ok] - 剩余数量：73
APS584  <->  [ok] - 剩余数量：72
APS586  <->  [ok] - 剩余数量：71
APS592  <->  [ok] - 剩余数量：70
APS583  <->  [ok] - 剩余数量：69
APS594  <->  [ok] - 剩余数量：68
APS595  <->  [ok] - 剩余数量：67
APS590  <->  [ok] - 剩余数量：66
APS591  <->  [ok] - 剩余数量：65
APS589  <->  [ok] - 剩余数量：64
APS596  <->  [ok] - 剩余数量：63
APS598  <->  [ok] - 剩余数量：62
APS597  <->  [ok] - 剩余数量：61
APS601  <->  [ok] - 剩余数量：60
APS605  <->  [ok] - 剩余数量：59
APS603  <->  [ok] - 剩余数量：58
APS608  <->  [ok] - 剩余数量：57
APS604  <->  [ok] - 剩余数量：56
APS599  <->  [ok] - 剩余数量：55
APS611  <->  [ok] - 剩余数量：54
APS606  <->  [ok] - 剩余数量：53
APS610  <->  [ok] - 剩余数量：52
APS615  <->  [ok] - 剩余数量：51
APS614  <->  [ok] - 剩余数量：50
APS619  <->  [ok] - 剩余数量：49
APS613  <->  [ok] - 剩余数量：48
APS617  <->  [ok] - 剩余数量：47
APS618  <->  [ok] - 剩余数量：46
APS622  <->  [ok] - 